In [ ]:
import requests
import json
import os
from datetime import datetime
import pandas as pd
from pandas import json_normalize
from io import StringIO
from shapely.geometry import Point, LineString
from dotenv import load_dotenv
load_dotenv(override=True)

/Users/kristian/Desktop/os2iot-lorawan-signal-quality/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

In [1]:
base_url = os.getenv('os2iot_BASE_URL')
os2iot_url = os.getenv('os2iot_URL')
os2iot_api_1 = os.getenv('os2iot_api_1')
org_id = os.getenv('os2iot_org_id')

url = f"{base_url}/chirpstack/gateway?organizationId={org_id}&limit=100000&offset=0"

headers = {
    'X-API-KEY': os2iot_api_1
}

response = requests.get(url, headers=headers)

data = json.loads(response.text)

for item in data['resultList']:
    location = item.get('location', {})
    item['latitude'] = location.get('latitude')
    item['longitude'] = location.get('longitude')

gateway_df = pd.DataFrame(data.get('resultList', []))[['gatewayId', 'name', 'description', 'operationalResponsibleName', 'rxPacketsReceived',
                                                       'txPacketsEmitted', 'modelName', 'antennaType', 
                                                       'placement', 'latitude', 'longitude','lastSeenAt', 'status']]

gateway_df = gateway_df.rename(columns={'gatewayId': 'gatewayID',
                                        'name': 'gateway_name',
                                        'latitude': 'gateway_latitude', 
                                        'longitude': 'gateway_longitude',
                                        'modelName': 'model_navn',
                                        'antennaType': 'antenne_type'})

gateway_df['lastSeenAt'] = pd.to_datetime(gateway_df['lastSeenAt'])
gateway_df['date'] = gateway_df['lastSeenAt'].dt.date

def extract_coordinates(point):
    return f"{point.y}, {point.x}"
gateway_df['gateway_point'] = gateway_df.apply(lambda row: Point(row['gateway_longitude'], row['gateway_latitude']), axis=1)
gateway_df['gateway_loc'] = gateway_df['gateway_point'].apply(extract_coordinates)
gateway_df = gateway_df.drop(columns=['lastSeenAt', 'gateway_point'])

gateway_df['url'] = gateway_df.apply(lambda row: f"{os2iot_url}/gateways/gateway-detail/{row['gatewayID']}", axis=1)

gateway_df.to_csv(os.getcwd() + '/gateways.csv', encoding='utf-8')

NameError: name 'os' is not defined

In [ ]:
gateway_df

In [ ]:
from pandas import json_normalize
df_api = json_normalize(data["resultList"])
df_api